In [1]:
import numpy as np
import pandas as pd
import requests 
import math
from scipy import stats
import xlsxwriter

In [20]:
stocks = pd.read_csv("sp_500_stocks.csv")
from API_token import IEX_CLOUD_API_TOKEN
def chunks(lst, n):
    # Yield successive n-sized chungs from list
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        
# IEX Cloud limits batch API calls to 100 tickers per request to boost up the time 
# Splitting into groups of 100 tickers
symbol_groups = list(chunks(stocks["Ticker"], 100))

symbol_strings = []
# creating strings of symbol for batch API calls
for i in range(len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

In [3]:
# PE ratio does not work well with negative earnings
# stocks buy back are difficult to value using price-to-book ratio
# Thus, composite basket of valuation metrics to build robust value strategy
# filter stocks with lowest percentiles on: PE ratio, Price-to-Book, Price-to-Sales, EV/EBITDA, and EV/GP

In [67]:
vi_column = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "PE ratio",
    "PE percentile",
    "Price-to-Book ratio",
    "Price-to-Book percentile",
    "Price-to-Sales ratio",
    "Price-to-Sales percentile",
    "EV/EBITDA ratio",
    "EV/EBITDA percentile",
    "EV/Gross Profit ratio",
    "EV/Gross Profit percentile",
    "Robust Value Score"
]

vi_df = pd.DataFrame(columns=vi_column)

for symbols in symbol_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=price,quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call).json()
    for symbol in symbols.split(","):
        ev = data[symbol]["advanced-stats"]["enterpriseValue"]
        ebitda = data[symbol]["advanced-stats"]["EBITDA"]
        grossProfit = data[symbol]["advanced-stats"]["grossProfit"]
        try:
            ev_to_ebitda = ev/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_grossProfit = ev/grossProfit
        except TypeError:
            ev_to_grossProfit = np.NaN
            
        vi_df = vi_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["price"],
                    "N/A",
                    # PE ratio
                    data[symbol]["quote"]['peRatio'],
                    "N/A",

                    # Price-to-Book
                    data[symbol]["advanced-stats"]["priceToBook"],
                    "N/A",
                    
                    # Price-to-Sales, 
                    data[symbol]["advanced-stats"]["priceToSales"],
                    "N/A",
                    
                    # EV/EBITDA
                    ev_to_ebitda,
                    "N/A",

                    # EV/GP
                    ev_to_grossProfit,
                    "N/A",
                    "N/A"
                ],
            index = vi_column
            ),
            ignore_index = True
        )
        
        
# fill na with mean value of that column
for column in ['PE ratio', 'Price-to-Book ratio','Price-to-Sales ratio', 'EV/EBITDA ratio', 'EV/Gross Profit ratio']:
    vi_df[column].fillna(vi_df[column].mean(), inplace = True)
    
# checking na value
vi_df[vi_df.isnull().any(axis=1)]





,Ticker,Price,Number of Shares to Buy,PE ratio,PE percentile,Price-to-Book ratio,Price-to-Book percentile,Price-to-Sales ratio,Price-to-Sales percentile,EV/EBITDA ratio,EV/EBITDA percentile,EV/Gross Profit ratio,EV/Gross Profit percentile,Robust Value Score


In [68]:
# filling out percentile

metrics = {
    "PE ratio": "PE percentile",
    "Price-to-Book ratio": "Price-to-Book percentile",
    "Price-to-Sales ratio": "Price-to-Sales percentile",
    "EV/EBITDA ratio": "EV/EBITDA percentile",
    "EV/Gross Profit ratio": "EV/Gross Profit percentile"
}


for metric in metrics.keys():
    for row in vi_df.index:
        vi_df.loc[row, metrics[metric]] = stats.percentileofscore(vi_df[metric], vi_df.loc[row, metric]) / 100
        
for row in vi_df.index:
    rvs = np.array([vi_df.loc[row, "PE percentile"], vi_df.loc[row, "Price-to-Book percentile"],
           vi_df.loc[row, "Price-to-Sales percentile"], vi_df.loc[row, "EV/EBITDA percentile"],
           vi_df.loc[row, "EV/Gross Profit percentile"]])
    vi_df.loc[row, "Robust Value Score"] = rvs.mean()

In [69]:
vi_df.sort_values("Robust Value Score", ascending=True, inplace=True)
vi_df = vi_df[:50]
vi_df.reset_index(drop = True, inplace=True)

def portfolio_input():
    global portfolio_size
    
    while True:
        # Enter integer value from the console.
        portfolio_size = input('Enter the value of your portfolio: ')

        try: 
            float(portfolio_size)
        except ValueError:
            print("Please enter a number")
        else:
            break
            
portfolio_input()
position_size = float(portfolio_size)/len(vi_df.index)
for i in vi_df.index:
    vi_df.loc[i, "Number of Shares to Buy"] = position_size//vi_df.loc[i, "Price"]

Enter the value of your portfolio: 10000000


In [70]:
vi_df

,Ticker,Price,Number of Shares to Buy,PE ratio,PE percentile,Price-to-Book ratio,Price-to-Book percentile,Price-to-Sales ratio,Price-to-Sales percentile,EV/EBITDA ratio,EV/EBITDA percentile,EV/Gross Profit ratio,EV/Gross Profit percentile,Robust Value Score
0,AAL,18.70,10695,-3.84,0.0555556,-1.6800,0.0555556,0.4900,0.0436508,-7.166205,0.0238095,1.416404,0.0535714,0.0464286
1,UNM,28.90,6920,7.48,0.113095,0.5363,0.0575397,0.4361,0.0357143,3.455354,0.0357143,0.434036,0.00793651,0.05
2,HPQ,40.32,4960,3.70,0.0734127,-13.2000,0.0376984,0.3288,0.0119048,4.016570,0.0456349,1.786581,0.0853175,0.0507937
3,PRU,121.90,1640,6.50,0.093254,0.7442,0.0654762,0.6518,0.0634921,4.868285,0.0634921,0.638393,0.0119048,0.0595238
4,KSS,49.20,4065,7.64,0.115079,1.3800,0.131944,0.3653,0.0198413,3.563010,0.0396825,1.116335,0.0376984,0.0688492
5,BEN,36.95,5412,5.05,0.0813492,0.8242,0.0734127,1.0900,0.131944,3.426249,0.0337302,0.901724,0.0238095,0.0688492
6,DHI,102.06,1959,4.46,0.077381,1.2100,0.106151,0.6595,0.0654762,3.492745,0.0376984,2.433835,0.119048,0.0811508
7,BA,233.36,857,-16.14,0.0436508,-9.4000,0.0436508,2.1100,0.330357,-35.615513,0.00595238,-1461.120582,0.00198413,0.085119
8,HPE,18.42,10857,3.55,0.0684524,0.5901,0.0615079,0.4099,0.0297619,7.694002,0.160714,2.173675,0.107143,0.0855159
9,AIZ,159.77,1251,7.15,0.103175,1.6100,0.183532,0.8946,0.10119,1.933028,0.0277778,0.859213,0.0218254,0.0875


In [71]:
# Writing to excel
writer = pd.ExcelWriter("RecommendedTrades.xlsx", engine="xlsxwriter")
vi_df.to_excel(writer, "RecommendedTrades", index = False)

# setting background and font color
background_color = "#0a0a23"
font_color = "#ffffff"

# String format for ticker
# $XX.XX format for stock price
# $XX,XXX format for market cap
# integer format for the number of shares purchase
string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

integer_format = writer.book.add_format(
    {
        "num_format": "0.0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

percent_format = writer.book.add_format(
    {
        "num_format": "0.0%",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

column_format = {
    "A": ["Ticker", string_format],
    "B": ["Price", dollar_format],
    "C": ["Number of Shares to Buy", integer_format],
    "D": ["PE ratio", integer_format],
    "E": ["PE percentile", percent_format],
    "F": ["Price-to-Book ratio", integer_format],
    "G": ["Price-to-Book percentile", percent_format],
    "H": ["Price-to-Sales ratio", integer_format],
    "I": ["Price-to-Sales percentile", percent_format],
    "J": ["EV/EBITDA ratio", integer_format],
    "K": ["EV/EBITDA percentile", percent_format],
    "L": ["EV/Gross Profit ratio", integer_format],
    "M": ["EV/Gross Profit percentile", percent_format],
    "N": ["Robust Value Score", percent_format],
}

# Formatting each column to its readable form 
# and reformat the column headers    
for column in column_format.keys():
    writer.sheets["RecommendedTrades"].set_column(f"{column}:{column}", 18, column_format[column][1]) # 18 is the pixel for the size of the column
    writer.sheets["RecommendedTrades"].write(f"{column}1", column_format[column][0], column_format[column][1])
    
writer.save()
